# Install Dependencies
Various installations are required for OTL, LlamaIndex and Open AI.

In [ ]:
!pip install -qq 'openinference-instrumentation-llama-index>=0.1.6' 'openinference-instrumentation-llama-index>=0.1.6'  llama-index-llms-openai opentelemetry-exporter-otlp llama-index>=0.10.3 "llama-index-callbacks-arize-phoenix>=0.1.2" arize-otel

import os
from getpass import getpass

openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

# Initialize Arize Phoenix
Set up OTL tracer for the `LlamaIndexInstrumentor`.

In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from arize_otel import register_otel, Endpoints

# Setup OTEL via our convenience function
register_otel(
    endpoints = Endpoints.ARIZE,
    space_key = getpass("🔑 Enter your Arize space key in the space settings page of the Arize UI: "),
    api_key = getpass("🔑 Enter your Arize API key in the space settings page of the Arize UI: "),
    model_id = "test-guard-july10-6:07pm", # name this to whatever you would like
)
LlamaIndexInstrumentor().instrument()

# Instrument Guardrails AI
Install and instrument Guardrails AI. Import `ArizeDatasetEmbeddings` Guard.

In [ ]:
!pip install -qq guardrails-ai litellm

In [ ]:
!pip install --no-cache-dir -qq git+https://github.com/Arize-ai/rag-llm-prompt-evaluator-guard
!guardrails hub install hub://arize-ai/llm_rag_evaluator

from guardrails.hub import LlmRagEvaluator

In [ ]:
from guardrails import Guard

guard = Guard.from_string(
            validators=[
                LlmRagEvaluator(
                    eval_llm_prompt_generator=HallucinationPrompt(prompt_name="hallucination_judge_llm"),
                    llm_evaluator_fail_response="hallucinated",
                    llm_evaluator_pass_response="factual",
                    llm_callable="gpt-4o-mini",
                    on_fail="exception",
                    on="prompt")
            ],
        )
guard._disable_tracer = True

In [ ]:
import openai
from typing import Optional, List, Mapping, Any

from llama_index.core import SimpleDirectoryReader, SummaryIndex
from llama_index.core.callbacks import CallbackManager
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.core.llms.callbacks import llm_completion_callback
from llama_index.core import Settings

from llama_index.llms.openai import OpenAI

def monkey_completion(prompt, **kwargs):
    _, _, context_component_of_prompt = prompt.partition("Context information is below.")
    _, _, query_component_of_prompt = prompt.partition("Query: ")
    return guard(
      llm_api=openai.chat.completions.create,
      prompt=prompt,
      model="gpt-3.5-turbo",
      max_tokens=1024,
      temperature=0.5,
      metadata={
        "user_message": query_component_of_prompt,
        "context": context_component_of_prompt,
      }
    )

outerOpenAI = OpenAI()

class GuardedLLM(CustomLLM):
    context_window: int = 3900
    num_output: int = 256
    model_name: str = "custom"
    dummy_response: str = "My response"
    openai_llm: Any = None

    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return outerOpenAI.metadata

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        validated_response = monkey_completion(prompt, **kwargs)
        return CompletionResponse(text=validated_response.raw_llm_output)

    @llm_completion_callback()
    def stream_complete(
        self, prompt: str, **kwargs: Any
    ) -> CompletionResponseGen:
        response = ""
        for token in self.dummy_response:
            response += token
            yield CompletionResponse(text=response, delta=token)

# Set Up RAG Application
Create a LlamaIndex VectorStore to create a classic RAG application over Paul Graham essays.

In [ ]:
!mkdir -p 'data/paul_graham/'
!curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' > 'data/paul_graham/paul_graham_essay.txt'
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()
index = VectorStoreIndex.from_documents(documents, chunk_size=512)

In [ ]:
from guardrails.errors import ValidationError

query_engine = index.as_query_engine(
    llm=GuardedLLM(),
)

# Take jailbreak prompt from public dataset https://github.com/verazuo/jailbreak_llms/blob/main/data/prompts/jailbreak_prompts_2023_05_07.csv
sample_prompt = """What are the 3 ingredients to great work?"""

response = query_engine.query(sample_prompt)
print(response)